<center><h1 style="font-size: 40px"> «Экстрактная суммаризация» </h1></center>
<center><h1 style="font-size: 30px"> Сравнение алгоритмов и моделей на разных метриках </h1></center>

# Содержание
### 0. [Используемые датасет и метрики](#chapter0)
### 1. [Алгоритмы экстрактной суммаризации из библиотеки sumy](#chapter1)
#### 1.1. [TextRank](#chapter1.1)
#### 1.2. [LexRank](#chapter1.2)
#### 1.3. [LSA](#chapter1.3)
#### 1.4. [KL Divergence](#chapter1.4)
#### 1.5. [Luhn](#chapter1.5)
### 2. [Предобученный ruBertSum](#chapter2)
### 3. [Результаты](#chapter3)

In [1]:
import numpy as np
import pandas as pd
import torch

# Библиотека с реализациями алгоритмов экстрактивной суммаризации
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

from transformers import BertForSequenceClassification, BertTokenizer, AutoConfig

# Предобученный на экстрактивную суммаризацию берт
from summarizer import Summarizer

from googletrans import Translator, LANGUAGES
from datasets import load_dataset

# Метрики
from rouge_score import rouge_scorer

In [2]:
import warnings
warnings.filterwarnings("ignore")

<center id="chapter0"><h1 style="font-size: 24px"> Используемые датасет и метрики </h1></center>

### Датасет: CNN-extractive -> https://huggingface.co/datasets/eReverter/cnn_dailymail_extractive?row=7

In [3]:
language = "russian" # Используемый язык, russian -> делать перевод

have_russian_file = True # True -> есть русский датасет в файле, False - загрузка датасета через load_dataset
path = "./russian_data.pkl" 

make_sample = True # True -> Взять подвыборку из датасета(особенно критично, если выбран русский, так как 20 строк переводит за минуту)
sample_len = 5 # Длина подвыборки

In [4]:
data = None
df = None

if have_russian_file:
    data = pd.read_pickle(path)
    df = pd.DataFrame(data[['ru_src', 'labels']].values, columns=['sentences', 'labels'])


else:
    dataset = load_dataset('eReverter/cnn_dailymail_extractive', split='validation')
    if make_sample:
        data = dataset.to_pandas().drop('tgt', axis=1).sample(sample_len, random_state=42)
    else:
        data = dataset.to_pandas().drop('tgt', axis=1)


    if language == 'english':
        data['src'] = data['src'].apply(lambda text: text.tolist())
        df = pd.DataFrame(data[['src', 'labels']].values, columns=['sentences', 'labels'])
    else:
        translator = Translator()
        data['src'] = data['src'].apply(lambda text: text.tolist())
        data['ru_src'] = data['src'].apply(lambda sentences: [translator.translate(sentence, src='en', dest='ru').text for sentence in sentences])

        df = pd.DataFrame(data[['ru_src', 'labels']].values, columns=['sentences', 'labels'])

In [5]:
def get_reference_summary(sentences, labels):
    """ 
    Получение образцовой экстрактивной суммаризации по массиву предложений и по маске входящих в суммаризацию предложений
    Параметры:
        sentences - массив строк(предложений)
        labels - маска предложений, 1 - предложение входит в суммаризацию, 0 - не входит
    Возвращает:
        строку
    """
    labels = [x==1 for x in labels]
    
    reference_summary = np.array(sentences)[labels]

    return ' '.join(reference_summary)

### Метрики

Будем использовать Rouge-метрики

In [6]:
#!pip install rouge-score
#!pip install evaluate
import rouge_score
import evaluate

In [7]:
# evaluate вроде бы позволяет считать метрики сразу для многих примеров, а не только для одного саммари
scorer = evaluate.load("rouge")

In [8]:
def get_rouge_scores(generated_summaries, reference_summaries):
    """
    Получение ROUGE-метрик для массива полученных суммаризаций
    Параметры:
        generated_summaries - массив полученных алгоритмом суммаризаций
        reference_summaries - массив соответствующих образцовых суммаризация
    Возвращает:
        словарь: ключ - метрика, значение - величина метрики
    """
    return scorer.compute(
        predictions=generated_summaries, 
        references=reference_summaries
    )

In [9]:
df

,sentences,labels
0,[(CNN) Палестинская администрация официально с...,"[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[(CNN) Не говоря уже о кошках, у которых девят...","[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,[(CNN) Если вы в последнее время следили за но...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[(CNN) Пятеро американцев, за которыми в течен...","[0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0]"
4,"[(CNN) Студент Дьюка признался, что повесил пе...","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
...,...,...
4995,[Немигающие глаза смотрят с обезглавленных гол...,"[1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4996,"[Приложение, которое позволяет пользователям н...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4997,"[Испанская больница утверждает, что провела «с...","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
4998,"[Ученые выявили новую сложную датировку., Мето...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ..."


In [10]:
# Дата фрейм, в который будем сохранять получающиеся суммаризации для каждой модели
# Первый столбец - образцовая суммаризация, с которой будем сравнивать
df_results = pd.DataFrame()
df_results["References"] = df[['sentences', 'labels']].apply(lambda cols: get_reference_summary(cols['sentences'], cols['labels']) , axis=1)
df_results

,References
0,Позже в том же месяце МУС начал предварительно...
1,"Ее взяла к себе жительница Мозес-Лейк, штат Ва..."
2,"Он, конечно, министр иностранных дел Ирана. В ..."
3,"В марте они заразились Эболой в Сьерра-Леоне, ..."
4,"(CNN) Студент Дьюка признался, что повесил пет..."
...,...
4995,Немигающие глаза смотрят с обезглавленных голо...
4996,"Приложение, которое позволяет пользователям на..."
4997,"Врачи заявили, что 27-часовая процедура восста..."
4998,Австралопитек Прометей жил примерно в то же вр...


In [11]:
# Все полученные значения метрик для всех алгоритмов будем хранить в датафрейме result_metrics

columns = ["rouge1", "rouge2", "rougeL", 'rougeLsum']
index = ["TextRank", "LexRank", "LSA", "KL", "Luhn", "ruBertSum"]

result_metrics = pd.DataFrame(columns=columns, index=index)

<center id="chapter1"><h1 style="font-size: 24px"> 1. Алгоритмы экстрактной суммаризации из библиотеки sumy </h1></center>

In [12]:
def get_parsed_text(text):
    """
    Парсит текст для алгоритмов из библиотеки sumy
    Параметры:
        text - одна строка
    """
    parser = PlaintextParser.from_string(text, Tokenizer(language))
    return parser.document

def get_summary(summarizer, sentences, labels):
    """
    Получение суммаризации при помощи алгоритмов из библиотеки sumy
    Параметры:
        summarizer - объект-суммаризатор, один из библиотеки sumy
        sentences - массив строк(предложений)
        labels - маска текста. Нужно, чтоб посчитать количество предложений, входящих в образцовую суммаризацию.
    Возвращает:
        строку - суммаризацию
    """
    text = ' '.join(sentences)
    parsed_text = get_parsed_text(text)
    summary = summarizer(parsed_text, int(sum(labels)))
    
    str_summary = ''
    for sentence in summary:
        str_summary+=str(sentence)  
    return str_summary

## 1.1. TextRank <a id="chapter1.1"></a>

In [13]:
from sumy.summarizers.text_rank import TextRankSummarizer

summarizer = TextRankSummarizer()

df_results['TextRank'] = df[['sentences', 'labels']].apply(lambda cols: get_summary(summarizer, cols['sentences'], cols['labels']), axis=1)

In [14]:
metrics_dict = get_rouge_scores(list(df_results['TextRank'].values), list(df_results["References"].values))
metrics_dict

{'rouge1': 0.21126236796504436,
 'rouge2': 0.10644054692734534,
 'rougeL': 0.2070681671019003,
 'rougeLsum': 0.2066887322169475}

In [15]:
for col in columns:
    result_metrics.loc["TextRank"][col] = metrics_dict[col]

## 1.2. LexRank <a id="chapter1.2"></a>

In [16]:
from sumy.summarizers.lex_rank import LexRankSummarizer

summarizer_lex = LexRankSummarizer()

df_results['LexRank'] = df[['sentences', 'labels']].apply(lambda cols: get_summary(summarizer_lex, cols['sentences'], cols['labels']), axis=1)

In [17]:
metrics_dict = get_rouge_scores(list(df_results['LexRank'].values), list(df_results["References"].values))
metrics_dict

{'rouge1': 0.23577795584900077,
 'rouge2': 0.11843411223228903,
 'rougeL': 0.23075763215107667,
 'rougeLsum': 0.23048015865028315}

In [18]:
for col in columns:
    result_metrics.loc["LexRank"][col] = metrics_dict[col]

## 1.3. LSA <a id="chapter1.3"></a>

In [19]:
from sumy.summarizers.lsa import LsaSummarizer

summarizer_lsa = LsaSummarizer()

df_results['LSA'] = df[['sentences', 'labels']].apply(lambda cols: get_summary(summarizer_lsa, cols['sentences'], cols['labels']), axis=1)

In [20]:
metrics_dict = get_rouge_scores(list(df_results['LSA'].values), list(df_results["References"].values))
metrics_dict

{'rouge1': 0.17382211322549318,
 'rouge2': 0.07880977141765376,
 'rougeL': 0.1707859184708569,
 'rougeLsum': 0.17072070001283238}

In [21]:
for col in columns:
    result_metrics.loc["LSA"][col] = metrics_dict[col]

## 1.4. KL Divergence <a id="chapter1.4"></a>

In [22]:
from sumy.summarizers.kl import KLSummarizer

summarizer_kl = KLSummarizer()

df_results['KL'] = df[['sentences', 'labels']].apply(lambda cols: get_summary(summarizer_kl, cols['sentences'], cols['labels']), axis=1)

In [23]:
metrics_dict = get_rouge_scores(list(df_results['KL'].values), list(df_results["References"].values))
metrics_dict

{'rouge1': 0.2034073291580784,
 'rouge2': 0.09986930246110058,
 'rougeL': 0.2001564415558217,
 'rougeLsum': 0.19963947500316334}

In [24]:
for col in columns:
    result_metrics.loc["KL"][col] = metrics_dict[col]

## 1.5. Luhn <a id="chapter1.5"></a>

In [25]:
from sumy.summarizers.luhn import LuhnSummarizer

summarizer_luhn = LuhnSummarizer()

df_results['Luhn'] = df[['sentences', 'labels']].apply(lambda cols: get_summary(summarizer_luhn, cols['sentences'], cols['labels']), axis=1)

In [26]:
metrics_dict = get_rouge_scores(list(df_results['Luhn'].values), list(df_results["References"].values))
metrics_dict

{'rouge1': 0.22938170405622646,
 'rouge2': 0.11306958886023438,
 'rougeL': 0.22520965815539917,
 'rougeLsum': 0.22504851552030047}

In [27]:
for col in columns:
    result_metrics.loc["Luhn"][col] = metrics_dict[col]

<center id="chapter2"><h1 style="font-size: 24px"> 2. Предобученный ruBertSum </h1></center>

In [28]:
# Объявляем модель, токенизатор, конфиг
ext_model_name = 'DeepPavlov/rubert-base-cased' # берем русский берт как основу суммаризатора
custom_tokenizer = BertTokenizer.from_pretrained(ext_model_name)
custom_config = AutoConfig.from_pretrained(ext_model_name)

# вывод скрытого состояния каждого слова = True. Это нужно, т.к. Summarizer работает именно с скрытыми состояниями (метаданными о тексте)
custom_config.output_hidden_states=True
custom_model = BertForSequenceClassification.from_pretrained(ext_model_name, config=custom_config)
bertsum_model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer) # задаём русский берт как модель и токенизатор у суммаризатора

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
df_results['BertSum'] = df[['sentences', 'labels']].apply(lambda cols: bertsum_model(' '.join(cols['sentences']), num_sentences=sum(cols['labels'])),
                                                          axis=1)

In [ ]:
metrics_dict = get_rouge_scores(list(df_results['BertSum'].values), list(df_results["References"].values))
metrics_dict

In [ ]:
for col in columns:
    result_metrics.loc["ruBertSum"][col] = metrics_dict[col]

<center id="chapter3"><h1 style="font-size: 24px"> 3. Результаты </h1></center>

In [ ]:
result_metrics

In [ ]:
result_metrics.to_csv("results.csv")